<a href="https://colab.research.google.com/github/Heeralall/cloud-computing-predict/blob/main/Data_deequ_tests_student_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processing Big Data - Deequ Analysis

© Explore Data Science Academy

## Honour Code
I {Koben Heeralall}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).
    Non-compliance with the honour code constitutes a material breach of contract.


## Context

Having completed manual data quality checks, it should be obvious that the process can become quite cumbersome. As the Data Engineer in the team, you have researched some tools that could potentially save the team from having to do this cumbersome work. In your research, you have come a across a tool called [Deequ](https://github.com/awslabs/deequ), which is a library for measuring the data quality of large datasets.

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://github.com/Explore-AI/Pictures/raw/master/data_engineering/transform/predict/DataQuality.jpg"
     alt="Data Quality"
     style="float: center; padding-bottom=0.5em"
     width=100%/>
     <p><em>Figure 1. Six dimensions of data quality</em></p>
</div>

You present this tool to your manager; he is quite impressed and gives you the go-ahead to use this in your implementation. You are now required to perform some data quality tests using this automated data testing tool.


In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()
!conda update -n base -c defaults conda -y
!conda create --name py37 python=3.7 -y

✨🍰✨ Everything looks OK!
Channels:
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.1.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - done

## Package Plan ##

  environment location: /usr/local/envs/py37

  added / updated specs:
    - python=3.7


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge 
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu 
  ca-certificates    conda-forge/linux-64::ca-certificates-2025.1.31-hbcca054_0 
  ld_impl_linux-64   conda-forge/linux-64::ld_impl_linux-64-2.43-h712a8e2_2 
  libffi             conda-forge/linux-64::libffi-3.4.6-h2dba641_0 
  libgcc             conda-forge/linux-64::

In [2]:
%%shell
   eval "$(conda shell.bash hook)"
   conda activate py37

In [3]:
"""
Colab is essentially running on a linux machine on Google Cloud Platform.
This means that should you want to install something in your notebook you
would have to run headless installs as well as wget. Copying the installs
below will ensure that you have spark and java installed in the environment
as well as available for the notebook.
"""
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz
!pip install -q findspark

In [4]:
"""
This section helps in rendering your notebook operable and ensuring that
your environment variables for spark are correct. Running this cell in any
notebook allows for any unresolved spark environment to be fixed.
"""
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"
os.environ["SPARK_VERSION"] = "3.0.0"

In [5]:
"""
We need to locate Spark in the system. For that, we import findspark and
use the findspark.init() method. If you want to know the location where
Spark is installed, use findspark.find()
"""
import findspark
findspark.init()
# findspark.find()
# import pyspark

In [4]:
"""
This cell is used to import data/files from your local machine to colab
"""
from google.colab import files
uploaded = files.upload()

Saving part-00000-52e1da7d-39e6-46bc-8167-46297b52948b-c000.snappy.parquet to part-00000-52e1da7d-39e6-46bc-8167-46297b52948b-c000.snappy.parquet
Saving part-00001-52e1da7d-39e6-46bc-8167-46297b52948b-c000.snappy.parquet to part-00001-52e1da7d-39e6-46bc-8167-46297b52948b-c000.snappy.parquet


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


> ## 🚩️ Important Notice 🚩️
>
>To successfully run `pydeequ` without any errors, please make sure that you have an environment that is running pyspark version 3.0.
> You are advised to **create a new conda environment** and install this specific version of pyspark to avoid any technical issues:
>
> `pip install pyspark==3.0`

<br>

## Import dependencies

If you do not have `pydeequ` already installed, install it using the following command:
- `pip install pydeequ`

In [1]:
!pip install pydeequ

In [2]:
!pip install pyspark==3.0.0 pydeequ==1.0.1 findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.7/204.7 MB 63.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044159 sha256=be3a600bac3f4056abc0adcca594fc52186f3a1e162da0c1c0b816ff5c7c1c83
  Stored in directory: /root/.cache/pip/wheels/56/d5/48/7beeda5357820f04fbe868a22e1d69fb922ec1c09b815bf6ae
Successfully built pyspark
  Attempting uninstall: pydeequ
    Found existing installation: pydeequ 1.4.0
    Uninstalling pydeequ-1.4.0:
      Successfully uninstalled pydeequ-1.4.0


In [7]:
!pip install --upgrade pydeequ
!pip install --upgrade pyspark

  Using cached pydeequ-1.4.0-py3-none-any.whl.metadata (9.4 kB)
Using cached pydeequ-1.4.0-py3-none-any.whl (37 kB)
  Attempting uninstall: pydeequ
    Found existing installation: pydeequ 1.0.1
    Uninstalling pydeequ-1.0.1:
      Successfully uninstalled pydeequ-1.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.4-py2.py3-none-any.whl size=317849766 sha256=f64dc26bacb7482399af07ec31a6395d52b070f0ee8e674d14105bb2f5ace82b
  Stored in directory: /root/.cache/pip/wheels/8d/28/22/5dbae8a8714ef046cebd320d0ef7c92f5383903cf854c15c0c
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9
    Uninstalling py4j-0.10.9:
      Successfully uninstalled py4j-0.10.9
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.0.0
    Uninstalling pyspark-3.0.0:
      Successfully uninstalled pyspark-3.0.0


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PyDeequTest").getOrCreate()

In [3]:
import os

# Set SPARK_VERSION environment variable

os.environ["SPARK_VERSION"] = "3.1"  # Replace with your Spark version

# Now import other libraries

import pyspark

import pydeequ

from pyspark.sql import SparkSession

# Initialize Spark session

spark = SparkSession.builder.appName("PyDeequTest").getOrCreate()



In [6]:
import os
os.environ["SPARK_VERSION"] = "3.1"  # Replace with your Spark version

# Now you can import other libraries or run your code:

import pyspark
# ... other imports ...

# Execute your script or other code:
# exec(open('your_script.py').read())

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pydeequ
from pydeequ.analyzers import *
from pydeequ.profiles import *
from pydeequ.suggestions import *
from pydeequ.checks import *
from pydeequ.verification import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType, DoubleType, IntegerType, DateType, NumericType, StructType, StringType, StructField

## Read data into spark dataframe

In this notebook, we set out to run some data quality tests, with the possiblity of running end to end on the years 1963, 1974, 1985, 1996, 2007, and 2018.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Data_ingestion_student_version.ipynb` notebook to create the parquet files for the following years:
>       - 1963
>       - 1974
>       - 1985
>       - 1996
>       - 2007
>       - 2018
>
>2. Ingest the data for the for the years given above. You should only do it one year at a time.
>3. Ingest the metadata file.


When developing your code, it will be sufficient to focus on a single year. However, after your development is done, you will need to run this notebook for all of the given years above so that you can answer all the questions given in the Data Testing MCQ.

In [ ]:
#TODO: Write your code here
# Use this variable (year) to determine which year your are focusing on
year = 1963


## **Run tests on the dataset**

## Test 1 - Null values ⛔️
For the first test, you are required to check the data for completeness.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check for missing values in the data.
>2. Display the results of your test.
>
> *You may use as many cells as necessary*


In [ ]:
#TODO: Write your code here

## Test 2 - Zero Values 🅾️

For the second test, you are required to check for zero values within the dataset.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check for zero values within the data.
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [ ]:
#TODO: Write your code here

## Test 3 - Negative values ➖️
The third test requires you to check that all values in the data are positive.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check negative values within the dataset.
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [ ]:
#TODO: Write your code here

## Test 4 - Determine Maximum Values ⚠️

For the fourth test, we want to find the maximum values in the dataset for the numerical fields. Extremum values can often be used to define an upper bound for the column values so we can define them as the threshold values.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Column Profiler Runner` to generate summary statistics for all the available columns.
>2. Extract the maximum values for all the numeric columns in the data.
>
> *You may use as many cells as necessary*

In [ ]:
#TODO: Write your code here

## Test 5 - Stock Tickers 💹️

For the fifth test, we want to determine if the stock tickers contained in our dataset are consistent. To do this, you will need to make use of use of the metadata file to check that the stock names used in the dataframe are valid.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to determine if the stock tickers contained in the dataset appear in the metadata file.
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [ ]:
#TODO: Write your code here

## Test 6 - Duplication 👥️
Lastly, we want to determine the uniqueness of the items found in the dataframe. You need to make use of the Verification Suite to check for the validity of the stock tickers.

Similar to the previous notebook - `Data_profiling_student_version.ipynb`, the first thing to check will be if the primary key values within the dataset are unique - in our case, that will be a combination of the stock name and the date. Secondly, we want to check if the entries are all unique, which is done by checking for duplicates across that whole dataset.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to determine the uniqueness of entries contained within the dataset.
>2. Display the results of your test.
>
> *You may use as many cells as necessary*



In [ ]:
#TODO: Write your code here